In [1]:
#!pip install -U accelerate bitsandbytes peft transformers datasets trl git-lfs wandb flash-attn sql-metadata

In [2]:
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from peft import PeftModel
from torch import cuda
from tqdm import tqdm
from modelscope import AutoModelForCausalLM, AutoTokenizer

/hpc2hdd/home/jzhao815/.conda/envs/dtssql/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-25 19:37:51,756 - modelscope - INFO - PyTorch version 2.2.2 Found.
2024-04-25 19:37:51,758 - modelscope - INFO - Loading ast index from /hpc2hdd/home/jzhao815/.cache/modelscope/ast_indexer
2024-04-25 19:37:52,036 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 2bbcf3979e8ce95c88f5a68d5ba83352 and a total number of 972 components indexed


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "qwen/CodeQwen1.5-7B-Chat"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2", # use with amper architecture
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config, # use when low on memory
    device_map = "auto"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.94s/it]


In [4]:
model = PeftModel.from_pretrained(model, "./final_checkpoint_codeQwen",torch_dtype = torch.bfloat16)
model = model.merge_and_unload()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [5]:
tokenizer.encode(' ;')

[5106]

In [6]:
from transformers import StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = [6203]):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids
    
def append_string_to_file(text, file_path):
  with open(file_path, 'a') as file:
      file.write(text + '\n')

def remove_spaces(text):
  return re.sub(r'\s+', ' ', text)

def call_mistral(inputs):
  output_tokens = model.generate(inputs, max_new_tokens=250, do_sample=False, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, stopping_criteria = [EosListStoppingCriteria()])
  return tokenizer.decode(output_tokens[0][len(inputs[0]):], skip_special_tokens=True)

In [7]:
df = pd.read_csv("useful_val_dataset3.csv")
results = []
for index, row in tqdm(df.iterrows(), total=len(df)):
  question = row['question']
  query = row['query']
  database_schema = row['filtered_database_schema']
  db_id = row['db_id']
  user_message = f"""Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question. Please pay special attention to the choice of table names and column names.
Put your answer inside the ```sql and ``` tags.
{database_schema}
###
Question: {question}
"""
  messages = [
      {"role": "user", "content": user_message.strip()}
  ]
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt",add_generation_prompt=True,tokenize = True).to(model.device)
  response = call_mistral(inputs)
  if ";" in response:
    response = response.split(";")[0]
    if "```sql" in response:
      response = response.split("```sql")[1]
  response = re.sub(r'\s+', ' ', response).strip()
  print("\n")
  print(response)
  print(query)
  print("============================")
  results.append([response, query,row['question'],row['db_id']])
  new_df = pd.DataFrame(results, columns = ['generated_query','reference_query','question','db_id'])

  0%|          | 0/1034 [00:00<?, ?it/s]/hpc2hdd/home/jzhao815/.conda/envs/dtssql/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  0%|          | 1/1034 [00:10<2:53:16, 10.06s/it]



SELECT COUNT(*) FROM singer
SELECT count(*) FROM singer


  0%|          | 2/1034 [00:17<2:23:20,  8.33s/it]



SELECT COUNT(*) FROM singer
SELECT count(*) FROM singer


  0%|          | 3/1034 [00:24<2:13:41,  7.78s/it]

...



SELECT feature_type_name FROM ref_feature_types WHERE feature_type_code = "AirCon"
SELECT T2.feature_type_name FROM Other_Available_Features AS T1 JOIN Ref_Feature_Types AS T2 ON T1.feature_type_code  =  T2.feature_type_code WHERE T1.feature_name  =  "AirCon"


100%|█████████▉| 1033/1034 [2:04:42<00:07,  7.30s/it]



SELECT t2.property_type_description FROM properties as t1 JOIN ref_property_types as t2 ON t1.property_type_code = t2.property_type_code GROUP BY t1.property_type_code
SELECT T2.property_type_description FROM Properties AS T1 JOIN Ref_Property_Types AS T2 ON T1.property_type_code  =  T2.property_type_code GROUP BY T1.property_type_code


100%|██████████| 1034/1034 [2:04:50<00:00,  7.24s/it]



SELECT property_name FROM properties WHERE property_type_code = "House" OR property_type_code = "Apartment" AND room_count > 1
SELECT property_name FROM Properties WHERE property_type_code  =  "House" UNION SELECT property_name FROM Properties WHERE property_type_code  =  "Apartment" AND room_count  >  1


In [8]:
new_df.to_csv('qwen_pred.csv')

In [9]:
predicted_queries = new_df["generated_query"]

with open("Predicted.sql", "w") as f:
    for query in predicted_queries:
        f.write(query + ";\n")

print("The generated query has been written to the Predicted.sql file.")

The generated query has been written to the Predicted.sql file.


In [1]:
import subprocess

# 假设 evaluation.py 在当前工作目录下，或者提供完整路径
script_path = 'eval/evaluation.py'
sql_file_path = 'Predicted.sql'

# 构建命令行命令
command = ['python', script_path, '--input', sql_file_path]

# 运行评估脚本
result = subprocess.run(command, capture_output=True, text=True)

# 打印输出结果
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: params as fllows 
 Namespace(input='Predicted.sql', gold='eval/eval_data/gold.txt', db='spider/database', table='eval/eval_data/tables.json', etype='all', plug_value=False, keep_distinct=False, progress_bar_for_each_datapoint=False, natsql=False, pred='Predicted.sql')
gseq_one length  1034
pseq_one length 1034
compare pred idx 0
compare pred idx 1
compare pred idx 2
compare pred idx 3
compare pred idx 4
compare pred idx 5
compare pred idx 6
compare pred idx 7
compare pred idx 8
compare pred idx 9
compare pred idx 10
compare pred idx 11
compare pred idx 12
compare pred idx 13
compare pred idx 14
compare pred idx 15
compare pred idx 16
medium pred: SELECT MAX(capacity) , AVG(capacity) FROM stadium;
medium gold: select max(capacity), average from stadium

compare pred idx 17
compare pred idx 18
compare pred idx 19
compare pred idx 20
compare pred idx 21
compare pred idx 22
medium pred: SELECT t2.name , COUNT(*) FROM concert as t1 JOIN stadium as t2 ON t1.stadium_id = t2.stadium_id